**DATACHALLENGE BDGIA DEBIASING MODEL**
---

In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import normalize
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

import torch
import torch.nn as nn
import torch.optim as optim

from evaluator_ANAELE import *

In [42]:
# FUNCTIONS

# to show performance

def evaluate(Y_pred,Y,S,will_print=1):
    '''returns model accuracy, final score, macro fscore ans TPR gap
    input : 2 np arrays of same dimension
    output : array of 4 values
    '''
    accuracy= accuracy_score(Y, Y_pred)  # Y_test are your original test labels
    print(f"Accuracy on transformed test data: {accuracy}")
    eval_scores, confusion_matrices_eval = gap_eval_scores(Y_pred, Y, S, metrics=['TPR'])
    final_score = (eval_scores['macro_fscore']+ (1-eval_scores['TPR_GAP']))/2

    if will_print==1:
        #print results
        print('final score',final_score)
        print('macro_fscore',eval_scores['macro_fscore'])
        print('1-eval_scores[\'TPR_GAP\']',1-eval_scores['TPR_GAP'])
    
    return accuracy, final_score, eval_scores['macro_fscore'],1-eval_scores['TPR_GAP'] , eval_scores , confusion_matrices_eval

# to predict X_test and save to file

def save_Y_pred_tofile(X, model,name):
    
    # save probabilities for each Xi (dim=28)
    y_pred_probs = model(X)
    probs=pd.DataFrame(y_pred_probs.detach().numpy(), columns= list(range(0,28)))
    file_name_probs = "y_pred_probs/y_pred_probs_"+str(name)+".csv"
    probs.to_csv(file_name_probs, header = None, index = None)

    # save predicted labels for each Xi (dim=1)
    y_pred = torch.argmax(y_pred_probs, dim=1)
    results=pd.DataFrame(y_pred.numpy(), columns= ['score'])
    file_name = "y_pred/Data_Challenge_"+str(name)+".csv"
    results.to_csv(file_name, header = None, index = None)

    return y_pred, y_pred_probs


##############################################################
#  DEFINE CUSTOM LOSS FUNCTION AND EVALUATION FUNCTIONS
#   
#   soft_f1_loss
#   macro_soft_f1_loss
#   calculate_exact_macro_f1
#   calculate_class_tpr_gap
#   average_tpr_gap_per_class
#   
##############################################################


def soft_f1_loss(y_true, y_pred):
    """
    Differentiable approximation of the F1 score as a loss function.
    """
    y_pred_probs = torch.softmax(y_pred, dim=1)
    tp = torch.sum(y_true * y_pred_probs, dim=0)
    pp = torch.sum(y_pred_probs, dim=0)
    ap = torch.sum(y_true, dim=0)
    precision = tp / (pp + 1e-6)
    recall = tp / (ap + 1e-6)
    soft_f1 = 2 * (precision * recall) / (precision + recall + 1e-6)
    loss = 1 - soft_f1.mean()  # Mean to aggregate over all classes
    return loss

def soft_macro_f1_loss(y_true, y_pred):
    """
    Differentiable approximation of the macro F1 score as a loss function.
    Calculates the F1 score for each class independently and then takes the average.
    """
    y_pred_probs = torch.softmax(y_pred, dim=1)
    tp = torch.sum(y_true * y_pred_probs, dim=0)
    pp = torch.sum(y_pred_probs, dim=0)
    ap = torch.sum(y_true, dim=0)
    
    precision = tp / (pp + 1e-6)
    recall = tp / (ap + 1e-6)
    
    f1_per_class = 2 * (precision * recall) / (precision + recall + 1e-6)
    macro_f1 = torch.mean(f1_per_class)  # Average F1 score across all classes
    
    loss = 1 - macro_f1  # Minimizing loss is maximizing macro F1 score
    return loss


def get_macro_f1(y_true, y_pred):
    """
    Calculate the exact macro F1 score for evaluation.
    Input : any format as tensors will be converted to Tensors of true label if dim >1 . Can be :
        - Tensor of probabilities(y_pred_probs) dimension (n,28)
        - Tensor of labels, one hote encoded (y_pred_one_hot) dimension (n,28)
        - Tensor of labels (y_pred_tensor) dimension (n,1)
    Ouput : scalar
    """
    #convert Tensors to 1 dimension (labels ranging from 0 to 27) if necessary
    y_pred_labels = torch.argmax(y_pred, dim=1) if y_pred.ndim > 1 else y_pred
    y_true_labels = torch.argmax(y_true, dim=1) if y_true.ndim > 1 else y_true

    " predict macro f1"
    f1 = f1_score(y_true_labels.cpu().numpy(), y_pred_labels.cpu().numpy(), average='macro')
    return f1

def get_tpr_gap(y_true, y_pred, protected_attribute, class_idx):
    """
    Calculate the TPR gap for a specific class across protected groups.
    
    Args:
    - y_true: Tensor of true labels, one-hot encoded.
    - y_pred_probs: Tensor of predicted probabilities (after softmax).
    - protected_attribute: Tensor indicating group membership for each instance.
    - class_idx: Index of the class for which to calculate the TPR gap.
    
    Returns:
    - TPR gap for the specified class.
    """
    #convert Tensors to 1 dimension (labels ranging from 0 to 27) if necessary
    y_pred_labels = torch.argmax(y_pred, dim=1) if y_pred.ndim > 1 else y_pred
    y_true_labels = torch.argmax(y_true, dim=1) if y_true.ndim > 1 else y_true
    
    # Calculate overall TPR for the current class
    overall_mask = y_true_labels == class_idx
    overall_tpr = torch.sum((y_pred_labels == class_idx) & overall_mask).float() / (torch.sum(overall_mask).float() + 1e-6)
    
    # Initialize list to store TPR for each protected group
    group_tprs = []
    
    # Calculate TPR for each protected group
    for group_val in protected_attribute.unique():
        group_mask = (protected_attribute == group_val) & overall_mask
        group_tpr = torch.sum((y_pred_labels == class_idx) & group_mask).float() / (torch.sum(group_mask).float() + 1e-6)
        group_tprs.append(group_tpr)
    
    # Calculate TPR gap for the current class
    tpr_gaps = torch.abs(torch.tensor(group_tprs) - overall_tpr)
    
    return torch.mean(tpr_gaps)  # Return the mean TPR gap for this class

def get_macro_tpr_gap(y_true, y_pred, protected_attribute):
    """
    Calculate the average TPR gap per class by calling tpr_gap for each class.
    
    Args:
    - y_true: Tensor of true labels, one-hot encoded.
    - y_pred: Tensor of predicted logits (before softmax).
    - protected_attribute: Tensor indicating group membership for each instance.
    
    Returns:
    - Average TPR gap across all classes.
    """
    #convert Tensors to 1 dimension (labels ranging from 0 to 27) if necessary
    y_pred_labels = torch.argmax(y_pred, dim=1) if y_pred.ndim > 1 else y_pred
    y_true_labels = torch.argmax(y_true, dim=1) if y_true.ndim > 1 else y_true
    
    # Initialize list to store TPR gaps for all classes
    class_tpr_gaps = []
    
    # Iterate over each class
    num_classes = len(y_true_labels.unique())
    for class_idx in range(num_classes):
        class_tpr_gap = get_tpr_gap(y_true_labels, y_pred_labels, protected_attribute, class_idx)
        class_tpr_gaps.append(class_tpr_gap)
    
    # Calculate the average TPR gap across all classes
    avg_tpr_gap = torch.mean(torch.stack(class_tpr_gaps))
    
    return avg_tpr_gap


def soft_final_score_loss(y_true, y_pred, protected_attribute):
    """
    Combine soft macro F1 score and TPR gap to create a final evaluation metric.
    """
    soft_macro_f1 = soft_macro_f1_loss(y_true, y_pred)  # Calculate soft macro F1 score
    macro_tpr_gap = get_macro_tpr_gap(y_true, y_pred, protected_attribute)  # Calculate TPR gap
    
    soft_final_score = ( soft_macro_f1 + (1 - macro_tpr_gap) ) / 2
    return soft_final_score

def get_final_score(y_true, y_pred, protected_attribute):
    """
    Combine soft macro F1 score and TPR gap to create a final evaluation metric.
    """
    #convert Tensors to 1 dimension (labels ranging from 0 to 27) if necessary
    y_pred_labels = torch.argmax(y_pred, dim=1) if y_pred.ndim > 1 else y_pred
    y_true_labels = torch.argmax(y_true, dim=1) if y_true.ndim > 1 else y_true

    macro_f1 = get_macro_f1(y_true_labels, y_pred_labels)  # Calculate macro F1 score
    macro_tpr_gap = get_macro_tpr_gap(y_true_labels, y_pred_labels, protected_attribute)  # Calculate macro TPR gap
    
    final_score = (macro_f1 + (1 - macro_tpr_gap)) / 2
    return final_score




In [26]:
##############################################################
# LOAD DATA, 
#############################################################

# Load pickle file and convert to numpy array
with open('data-challenge-student.pickle', 'rb') as handle:
    # dat = pickle.load(handle)
    dat = pd.read_pickle(handle)
 
#Check keys()
print(dat.keys())
X = dat['X_train']
Y = dat['Y']
S = dat['S_train']

#create a label to distiguish 56 labels Y x 2 (man or woman)
# 0 to 27 = non sensitive group | 28 + [0 , 27] = 28 to 55 = sensitive group
Y56 = Y+28*S

X_test_true = dat['X_test']
S_test_true = dat['S_test']

# check size
print(X.shape,Y.shape,S.shape,X_test_true.shape,S_test_true.shape)


dict_keys(['X_train', 'X_test', 'Y', 'S_train', 'S_test'])
(27749, 768) (27749,) (27749,) (11893, 768) (11893,)


In [27]:
##############################################################
# train_test_split (np.arrays)
##############################################################

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, Y56_train, Y56_test = train_test_split(X, Y56, test_size=0.2, random_state=42)
Y_train = Y56_train % 28  # reste (original Y)   ex 33% 28 = classe 5 
S_train = Y56_train//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)
Y_test = Y56_test % 28  # reste (original Y)   ex 33% 28 = classe 5 
S_test = Y56_test//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)

# impression des dimensions
print('train:',X_train.shape,Y_train.shape,S_train.shape)
print('test:',X_test.shape,Y_test.shape, S_test.shape)

##############################################################
# 1. Transform DataFrames into Tensors
##############################################################

X_tensor = torch.tensor(X.values, dtype=torch.float32)
Y_tensor = torch.tensor(Y.values, dtype=torch.long)
S_tensor = torch.tensor(S.values, dtype=torch.long)

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train.values, dtype=torch.long)
S_train_tensor = torch.tensor(S_train.values, dtype=torch.long)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test.values, dtype=torch.long)
S_test_tensor = torch.tensor(S_test.values, dtype=torch.long)

Y_train_one_hot = torch.nn.functional.one_hot(Y_train_tensor, num_classes=Y_train.nunique())
Y_test_one_hot = torch.nn.functional.one_hot(Y_test_tensor, num_classes=Y_train.nunique())

X_test_true_tensor = torch.tensor(X_test_true.values, dtype=torch.float32)

# impression des dimensions
print('train_tensor:',X_train_tensor.shape,Y_train_tensor.shape,S_train_tensor.shape, type(X_train_tensor))
print('test_tensor:',X_test_tensor.shape,Y_test_tensor.shape, S_test_tensor.shape, type(X_test_tensor))
print('Y_train_one_hot:',Y_train_one_hot.shape, type(Y_train_one_hot))
print('X_test_true_tensor:',X_test_true_tensor.shape, type(X_test_true_tensor))

train: (22199, 768) (22199,) (22199,)
test: (5550, 768) (5550,) (5550,)
train_tensor: torch.Size([22199, 768]) torch.Size([22199]) torch.Size([22199]) <class 'torch.Tensor'>
test_tensor: torch.Size([5550, 768]) torch.Size([5550]) torch.Size([5550]) <class 'torch.Tensor'>
Y_train_one_hot: torch.Size([22199, 28]) <class 'torch.Tensor'>
X_test_true_tensor: torch.Size([11893, 768]) <class 'torch.Tensor'>


**NN with customized loss function (final score)**
---

In [27]:
# 1. Define the model and optimizer
# ---------------------------------

model = nn.Sequential(
    nn.Linear(768, 28),  # Assuming 768 input features and 28 classes
    nn.ReLU(),  # Adding a ReLU activation function
    nn.Linear(28, 28),  # Additional layer for complexity
    nn.LogSoftmax(dim=1)  # LogSoftmax for multi-class classification
)

optimizer = optim.SGD(model.parameters(), lr=0.1)

# 2. Train the model with the custom loss function final_eval
# -----------------------------------------------------------

num_epochs = 1000  # Adjust as necessary

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    outputs_train = model(X_train_tensor)
    
    #loss = soft_final_score_loss(Y_train_one_hot.float(), outputs_train, S_train_tensor)
    loss = soft_macro_f1_loss(Y_train_one_hot.float(), outputs_train)*10 + get_macro_tpr_gap(Y_train_one_hot.float(), outputs_train, S_train_tensor )

    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 100 == 0:
        model.eval()
        with torch.no_grad():
            # Calculate metrics for training data
            outputs_train = model(X_train_tensor) # probabilities

            # Evaluate predictions on training data
            final_score_train = get_final_score(Y_train_tensor, outputs_train, S_train_tensor)
            macro_f1_train = get_macro_f1(Y_train_tensor, outputs_train)
            inv_macro_tpr_gap_train = 1 - get_macro_tpr_gap(Y_train_tensor, outputs_train, S_train_tensor)
            
            # Calculate metrics for test data
            outputs_test = model(X_test_tensor)

            # Evaluate predictions on training data
            final_score_test = get_final_score(Y_test_tensor, outputs_test, S_test_tensor)
            macro_f1_test = get_macro_f1(Y_test_tensor, outputs_test)
            inv_macro_tpr_gap_test = 1 - get_macro_tpr_gap(Y_test_tensor, outputs_test, S_test_tensor)
            
            print(f'Epoch {epoch + 1}, Loss: {loss.item()}, Final Score Train: {final_score_train.item()}, Final Score Test: {final_score_test.item()}, macro F1 Train: {macro_f1_train}, macro F1 Test: {macro_f1_test}, 1-TPR Gap Train: {inv_macro_tpr_gap_train}, 1-TPR Gap Test: {inv_macro_tpr_gap_test}')
            # print(f'Epoch {epoch + 1}, Loss: {loss.item()},  macro F1 Train: {macro_f1_train}, macro F1 Test: {macro_f1_test}')# Final Score Train: {final_score_train.item()}, Final Score Test: {final_score_test.item()}, macro F1 Train: {macro_f1_train}, macro F1 Test: {macro_f1_test}, 1-TPR Gap Train: {inv_macro_tpr_gap_train}, 1-TPR Gap Test: {inv_macro_tpr_gap_test}')


# 4. Make Predictions and Evaluate with final_score
# -------------------------------------------------
            
with torch.no_grad():
    model.eval()
    Y_pred_probs = model(X_test_tensor) # dim = 28
    Y_pred_tensor = torch.argmax(Y_pred_probs, dim=1)  # dim = 1 (Get the class with the highest probability)
    Y_pred_one_hot = torch.nn.functional.one_hot(Y_pred_tensor, num_classes=28)  # dim = 28
 
    macro_f1 = get_macro_f1(Y_test_tensor, Y_pred_tensor)
    inv_macro_tpr_gap = 1 - get_macro_tpr_gap(Y_test_tensor, Y_pred_probs, S_test_tensor)
    final_score = get_final_score(Y_test_tensor, Y_pred_probs, S_test_tensor)
    print(f'Final Evaluation Score: {final_score.item()} Macro F1: {macro_f1.item()} 1-TPR_gap: { inv_macro_tpr_gap.item() }')

train_tensor: torch.Size([22199, 768]) torch.Size([22199]) torch.Size([22199]) <class 'torch.Tensor'>
test_tensor: torch.Size([5550, 768]) torch.Size([5550]) torch.Size([5550]) <class 'torch.Tensor'>
Y_train_one_hot torch.Size([22199, 28]) <class 'torch.Tensor'>
Epoch 100, Loss: 9.229928970336914, Final Score Train: 0.5473254323005676, Final Score Test: 0.5484995245933533, macro F1 Train: 0.10194340859892699, macro F1 Test: 0.10541046804592533, 1-TPR Gap Train: 0.9927074313163757, 1-TPR Gap Test: 0.9915885925292969
Epoch 200, Loss: 8.115486145019531, Final Score Train: 0.6071764826774597, Final Score Test: 0.6037586331367493, macro F1 Train: 0.2593545053976912, macro F1 Test: 0.2511065513502886, 1-TPR Gap Train: 0.9549984335899353, 1-TPR Gap Test: 0.9564107060432434
Epoch 300, Loss: 6.555143356323242, Final Score Train: 0.6743472218513489, Final Score Test: 0.6614153981208801, macro F1 Train: 0.4129127061666681, macro F1 Test: 0.39077676498786784, 1-TPR Gap Train: 0.935781717300415, 1-

In [11]:
name = 'test'
save_Y_pred_tofile(X_test_true_tensor, model,name)

NameError: name 'y_pred' is not defined

In [29]:
# Convert Y_pred to a DataFrame
Y_pred_df = pd.DataFrame(Y_pred_tensor.numpy(), columns=['Predicted'])

# Evaluate Y_pred compared to Y_test (assuming Y_test is a numpy array or a pandas Series)
print(classification_report(Y_test, Y_pred_df['Predicted']))

              precision    recall  f1-score   support

           0       0.66      0.62      0.64        81
           1       0.70      0.57      0.63       127
           2       0.81      0.87      0.84       458
           3       0.00      0.00      0.00        36
           4       0.89      0.65      0.75        48
           5       0.81      0.78      0.79        72
           6       0.84      0.71      0.77       178
           7       0.75      0.74      0.75        54
           8       0.92      0.67      0.77        18
           9       0.79      0.81      0.80        91
          10       0.77      0.45      0.57        22
          11       0.55      0.83      0.66       286
          12       0.86      0.70      0.77       110
          13       0.74      0.73      0.73       258
          14       0.86      0.70      0.77       112
          15       0.00      0.00      0.00        19
          16       0.48      0.48      0.48        33
          17       0.74    

/home/anaele/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/anaele/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/anaele/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [38]:
Y_pred_probs = model(X_test_true_tensor)
Y_pred_tensor = torch.argmax(Y_pred_probs, dim=1)

results=pd.DataFrame(Y_pred_tensor, columns= ['score'])
name = 'NN_with_custom_loss'
file_name = "Data_Challenge_MDI_341_"+str(name)+".csv"
results.to_csv(file_name, header = None, index = None)


**ARRET ANTICIPE DU NN**
---

In [ ]:
        val_loss = soft_macro_f1_loss(Y_val_one_hot.float(), outputs_val)*10 + get_macro_tpr_gap(Y_val_one_hot.float(), outputs_val, S_val_tensor)  # Assurez-vous d'avoir Y_val_one_hot et S_val_tensor
        
    # Vérifier si la perte de validation s'est améliorée
    if best_loss is None or val_loss < best_loss:
        best_loss = val_loss
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print(f'Arrêt précoce après {epoch+1} époques')
            break  # Arrêter l'entraînement

In [47]:
# 1. Define the model and optimizer
# ---------------------------------

model = nn.Sequential(
    nn.Linear(768, 28),  # Assuming 768 input features and 28 classes
    nn.ReLU(),  # Adding a ReLU activation function
    nn.Linear(28, 28),  # Additional layer for complexity
    nn.LogSoftmax(dim=1)  # LogSoftmax for multi-class classification
)

optimizer = optim.Adam(model.parameters(), lr=0.01)


# 2. Paramètres pour l'arrêt précoce
# -------------------------------
patience = 10  # Nombre d'époques à attendre après la dernière amélioration de la perte de validation
best_loss = None
epochs_without_improvement = 0

# 3. Train the model with the custom loss function final_eval
# -----------------------------------------------------------
num_epochs = 10000  # Adjust as necessary

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    outputs_train = model(X_train_tensor)
    
    #loss = soft_final_score_loss(Y_train_one_hot.float(), outputs_train, S_train_tensor)
    loss = soft_macro_f1_loss(Y_train_one_hot.float(), outputs_train)*alpha + get_macro_tpr_gap(Y_train_one_hot.float(), outputs_train, S_train_tensor )

    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 100 == 0:
        model.eval()
        with torch.no_grad():
            # Calculate metrics for training data
            outputs_train = model(X_train_tensor) # probabilities

            # Evaluate predictions on training data
            final_score_train = get_final_score(Y_train_tensor, outputs_train, S_train_tensor)
            macro_f1_train = get_macro_f1(Y_train_tensor, outputs_train)
            inv_macro_tpr_gap_train = 1 - get_macro_tpr_gap(Y_train_tensor, outputs_train, S_train_tensor)

            # Calculate metrics for test data
            outputs_test = model(X_test_tensor)

            # Evaluate predictions on training data
            final_score_test = get_final_score(Y_test_tensor, outputs_test, S_test_tensor)
            macro_f1_test = get_macro_f1(Y_test_tensor, outputs_test)
            inv_macro_tpr_gap_test = 1 - get_macro_tpr_gap(Y_test_tensor, outputs_test, S_test_tensor)

            # Vérifier si la perte de validation s'est améliorée (arret précoce)
            if best_loss is None or final_score_test < best_loss:
                best_loss = final_score_test
                epochs_without_improvement = 0
            else:
                epochs_without_improvement += 1
                if epochs_without_improvement >= patience:
                    print(f'Arrêt précoce après {epoch+1} époques')
                    break  # Arrêter l'entraînement

            print(f'Epoch {epoch + 1}, Loss: {loss.item()}, Final Score Train: {final_score_train.item()}, Final Score Test: {final_score_test.item()}, macro F1 Train: {macro_f1_train}, macro F1 Test: {macro_f1_test}, 1-TPR Gap Train: {inv_macro_tpr_gap_train}, 1-TPR Gap Test: {inv_macro_tpr_gap_test}')
            # print(f'Epoch {epoch + 1}, Loss: {loss.item()},  macro F1 Train: {macro_f1_train}, macro F1 Test: {macro_f1_test}')# Final Score Train: {final_score_train.item()}, Final Score Test: {final_score_test.item()}, macro F1 Train: {macro_f1_train}, macro F1 Test: {macro_f1_test}, 1-TPR Gap Train: {inv_macro_tpr_gap_train}, 1-TPR Gap Test: {inv_macro_tpr_gap_test}')


# 4. Make Predictions and Evaluate with final_score
# -------------------------------------------------
            
with torch.no_grad():
    model.eval()
    Y_pred_probs = model(X_test_tensor) # dim = 28
    Y_pred_tensor = torch.argmax(Y_pred_probs, dim=1)  # dim = 1 (Get the class with the highest probability)
    Y_pred_one_hot = torch.nn.functional.one_hot(Y_pred_tensor, num_classes=28)  # dim = 28
 
    macro_f1 = get_macro_f1(Y_test_tensor, Y_pred_tensor)
    inv_macro_tpr_gap = 1 - get_macro_tpr_gap(Y_test_tensor, Y_pred_probs, S_test_tensor)
    final_score = get_final_score(Y_test_tensor, Y_pred_probs, S_test_tensor)
    print(f'Final Evaluation Score: {final_score.item()} Macro F1: {macro_f1.item()} 1-TPR_gap: { inv_macro_tpr_gap.item() }')

Epoch 100, Loss: 0.980758786201477, Final Score Train: 0.8257572054862976, Final Score Test: 0.7689204216003418, macro F1 Train: 0.7131982118809204, macro F1 Test: 0.6150132138136468, 1-TPR Gap Train: 0.9383162260055542, 1-TPR Gap Test: 0.9228276014328003
Epoch 200, Loss: 0.8239663243293762, Final Score Train: 0.8505457639694214, Final Score Test: 0.7687321901321411, macro F1 Train: 0.7565347517471814, macro F1 Test: 0.6086528022331705, 1-TPR Gap Train: 0.9445567727088928, 1-TPR Gap Test: 0.9288116097450256
Epoch 300, Loss: 0.7659595012664795, Final Score Train: 0.8582633137702942, Final Score Test: 0.7669168710708618, macro F1 Train: 0.7717500860280669, macro F1 Test: 0.6037370920367443, 1-TPR Gap Train: 0.9447765350341797, 1-TPR Gap Test: 0.9300966262817383
Epoch 400, Loss: 0.7373118996620178, Final Score Train: 0.8617070913314819, Final Score Test: 0.7658792734146118, macro F1 Train: 0.7784565046319429, macro F1 Test: 0.5997586318884097, 1-TPR Gap Train: 0.9449577331542969, 1-TPR Ga

**FUNCTION FOR NN WITH CUSTOM LOSS**
---

In [55]:
def train_NN_with_custom_loss(model, optimizer, alpha, X_train_tensor, Y_train_tensor, S_train_tensor, X_test_tensor, Y_test_tensor, S_test_tensor):

    # 2. Paramètres pour l'arrêt précoce
    # -------------------------------
    patience = 10  # Nombre d'époques à attendre après la dernière amélioration de la perte de validation
    best_loss = None
    epochs_without_improvement = 0

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
    
        outputs_train = model(X_train_tensor)
    
        #loss = soft_final_score_loss(Y_train_one_hot.float(), outputs_train, S_train_tensor)
        loss = alpha * soft_macro_f1_loss(Y_train_one_hot.float(), outputs_train) + get_macro_tpr_gap(Y_train_one_hot.float(), outputs_train, S_train_tensor )

        loss.backward()
        optimizer.step()

        model.eval()
        
        # 1. Vérifier si la perte de validation s'est améliorée (arret précoce)

        # Calculate metrics for test data
        outputs_test = model(X_test_tensor)

        # Evaluate predictions on test (validation) data
        final_score_test = get_final_score(Y_test_tensor, outputs_test, S_test_tensor)
        outputs_test = model(X_test_tensor)
        
        if best_loss is None or final_score_test < best_loss:
            best_loss = final_score_test
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print(f'Arrêt précoce après {epoch+1} époques')
                break  # Arrêter l'entraînement

        # 2. Impression de l'apprentissage et des scores train et test
        if (epoch + 1) % 100 == 0:
            
            with torch.no_grad():
                # Calculate metrics for training data
                outputs_train = model(X_train_tensor) # probabilities
                # Evaluate predictions on training data
                final_score_train = get_final_score(Y_train_tensor, outputs_train, S_train_tensor)
                macro_f1_train = get_macro_f1(Y_train_tensor, outputs_train)
                inv_macro_tpr_gap_train = 1 - get_macro_tpr_gap(Y_train_tensor, outputs_train, S_train_tensor)
            
                # Calculate metrics for test data
                outputs_test = model(X_test_tensor)
                # Evaluate predictions on training data
                final_score_test = get_final_score(Y_test_tensor, outputs_test, S_test_tensor)
                macro_f1_test = get_macro_f1(Y_test_tensor, outputs_test)
                inv_macro_tpr_gap_test = 1 - get_macro_tpr_gap(Y_test_tensor, outputs_test, S_test_tensor)

                print(f'Epoch {epoch + 1}, Loss: {loss.item()}, Final Score Train: {final_score_train.item()}, Final Score Test: {final_score_test.item()}, macro F1 Train: {macro_f1_train}, macro F1 Test: {macro_f1_test}, 1-TPR Gap Train: {inv_macro_tpr_gap_train}, 1-TPR Gap Test: {inv_macro_tpr_gap_test}')
            
    # 4. Make Predictions and Evaluate with final_score
    # -------------------------------------------------
            
    with torch.no_grad():
        model.eval()
        Y_pred_probs = model(X_test_tensor) # dim = 28 (Probabilities for each class)
        Y_pred_tensor = torch.argmax(Y_pred_probs, dim=1)  # dim = 1 (Get the class with the highest probability)
 
        macro_f1 = get_macro_f1(Y_test_tensor, Y_pred_tensor)
        inv_macro_tpr_gap = 1 - get_macro_tpr_gap(Y_test_tensor, Y_pred_probs, S_test_tensor)
        final_score = get_final_score(Y_test_tensor, Y_pred_probs, S_test_tensor)
        print(f'Final Evaluation Score: {final_score.item()} Macro F1: {macro_f1.item()} 1-TPR_gap: { inv_macro_tpr_gap.item() }')

    return model, Y_pred_probs, Y_pred_tensor, final_score, macro_f1, inv_macro_tpr_gap

In [56]:
#################################################
#          TEST DES PARAMETRES
################################################


# 1. Define the model and optimizer and train
# --------------------------------------------------

model = nn.Sequential(
    nn.Linear(768, 28),  # Assuming 768 input features and 28 classes
    nn.LogSoftmax(dim=1))  # LogSoftmax for multi-class classification

learning_rate=0.01
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
num_epochs = 10000 

# 2. Train the model with the custom loss function final_eval
# -----------------------------------------------------------
name = 'NN-28-28_Adam'+'_lr_'+str(learning_rate)
print('\n\n Starting to train model', name)
model_trained, Y_pred_probs, Y_pred_tensor, final_score, macro_f1, inv_macro_tpr_gap = train_NN_with_custom_loss(model,optim.Adam(model.parameters(), lr=learning_rate) , 5, X_train_tensor, Y_train_tensor, S_train_tensor, X_test_tensor, Y_test_tensor, S_test_tensor)
Res.loc[i]=[name,optimizer,learning_rate,alpha,final_score, macro_f1, inv_macro_tpr_gap]
save_Y_pred_tofile(X_test_true_tensor, model_trained,name)




 Starting to train model NN-28-28_Adam_lr_0.01


Arrêt précoce après 12 époques
Final Evaluation Score: 0.6582198739051819 Macro F1: 0.4365978290566592 1-TPR_gap: 0.8798419237136841


(tensor([18, 21, 11,  ..., 21,  2, 19]),
 tensor([[-9.6625e+00, -7.2440e+00, -9.7094e+00,  ..., -1.0444e+01,
          -6.8781e+00, -7.8592e+00],
         [-6.5108e+00, -5.8126e+00, -5.0276e+00,  ..., -2.4211e+00,
          -7.0888e+00, -8.4636e+00],
         [-3.8640e+00, -3.9272e+00, -5.2261e+00,  ..., -6.0567e+00,
          -2.4829e+00, -4.7992e+00],
         ...,
         [-7.6490e+00, -1.0700e+01, -4.2136e+00,  ..., -5.2082e+00,
          -8.0097e+00, -5.9439e+00],
         [-6.7491e+00, -7.8273e+00, -2.8641e-03,  ..., -8.2286e+00,
          -9.9370e+00, -9.9463e+00],
         [-5.8481e+00, -8.0534e+00, -3.7037e+00,  ..., -6.3331e+00,
          -7.2380e+00, -4.9324e+00]], grad_fn=<LogSoftmaxBackward0>))

In [57]:
#################################################
#          BOUCLE HYPERPARAMETRES
################################################


# 1. Define the model and optimizer and train
# --------------------------------------------------

model = nn.Sequential(
    nn.Linear(768, 28),  # Assuming 768 input features and 28 classes
    nn.ReLU(),  # Adding a ReLU activation function
    nn.Linear(28, 28),  # Additional layer for complexity
    nn.LogSoftmax(dim=1))  # LogSoftmax for multi-class classification

learning_rate=0.01
optimizer_dict = {'Momentum' : optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9),
                'NAG': optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True),
                'Adam': optim.Adam(model.parameters(), lr=learning_rate),
                'Adagrad': optim.Adagrad(model.parameters(), lr=learning_rate, lr_decay=0, weight_decay=0, initial_accumulator_value=0, eps=1e-10),
                 'SGD': optim.SGD(model.parameters(), lr=learning_rate)
                }
lr_list = [0.1, 0.05, 0.01, 0.005, 0.001]
num_epochs = 10000 

# 2. Train the model with the custom loss function final_eval
# -----------------------------------------------------------
Res=pd.DataFrame(columns=['model','optimizer','lr','alpha','final_score','macro_f1','macro_tpr_gap'])
i=0
for opt_name, optimizer in optimizer_dict.items():
    for learning_rate in lr_list:
        name = opt_name+'_lr_'+str(learning_rate)+'_alpha_'+str(i)
        print('\n\n Starting to train model', name)
        model_trained, Y_pred_probs, Y_pred_tensor, final_score, macro_f1, inv_macro_tpr_gap = train_NN_with_custom_loss(model, optimizer, alpha, X_train_tensor, Y_train_tensor, S_train_tensor, X_test_tensor, Y_test_tensor, S_test_tensor)
        Res.loc[i]=[name,optimizer,learning_rate,alpha,final_score, macro_f1, inv_macro_tpr_gap]
        save_Y_pred_tofile(X_test_true_tensor, model_trained,name)
        i+=1




 Starting to train model Momentum_lr_0.1_alpha_0
Arrêt précoce après 11 époques
Final Evaluation Score: 0.5008360147476196 Macro F1: 0.0068063568906823494 1-TPR_gap: 0.9948656558990479


 Starting to train model Momentum_lr_0.05_alpha_1
Arrêt précoce après 11 époques
Final Evaluation Score: 0.5026751160621643 Macro F1: 0.0054541114691490635 1-TPR_gap: 0.9998961687088013


 Starting to train model Momentum_lr_0.01_alpha_2
Arrêt précoce après 29 époques
Final Evaluation Score: 0.5027225613594055 Macro F1: 0.005445120791325851 1-TPR_gap: 1.0


 Starting to train model Momentum_lr_0.005_alpha_3
Arrêt précoce après 11 époques
Final Evaluation Score: 0.5027225613594055 Macro F1: 0.005445120791325851 1-TPR_gap: 1.0


 Starting to train model Momentum_lr_0.001_alpha_4
Arrêt précoce après 11 époques
Final Evaluation Score: 0.5027225613594055 Macro F1: 0.005445120791325851 1-TPR_gap: 1.0


 Starting to train model NAG_lr_0.1_alpha_5
Arrêt précoce après 11 époques
Final Evaluation Score: 0.5027

In [ ]:
#ENTRAINEMENT SUR TOUT LE MODELE

# 2. Train the model with the custom loss function final_eval
# -----------------------------------------------------------
Res=pd.DataFrame(columns=['model','optimizer','lr','alpha','final_score','macro_f1','macro_tpr_gap'])
i=0
for opt_name, optimizer in optimizer_dict.items():
    for learning_rate in lr_list:
        for i in range(1,10):
            alpha=i
            name = 'all'+opt_name+'_lr_'+str(learning_rate)+'_alpha_'+str(i)
            print('\n\n Starting to train model', name)
            model_trained, Y_pred_probs, Y_pred_tensor, final_score, macro_f1, inv_macro_tpr_gap = train_NN_with_custom_loss(model, optimizer, alpha, X_tensor, Y_tensor, S_tensor, X_test_tensor, Y_test_tensor, S_test_tensor)
            Res.loc[i]=[name,optimizer,learning_rate,alpha,final_score, macro_f1, inv_macro_tpr_gap]
            save_Y_pred_tofile(X_test_true_tensor, model_trained,name)
            i+=1

In [ ]:
model = nn.Sequential(
    nn.Linear(768, 2048),  # Couche d'entrée à la première couche cachée
    nn.ReLU(),  # Fonction d'activation ReLU
    nn.Dropout(p=0.5),  # Dropout avec une probabilité de désactivation de 50%
    nn.Linear(2048, 512),  # De la première couche cachée à la deuxième couche cachée
    nn.ReLU(),  # Une autre fonction d'activation ReLU après la deuxième couche cachée
    nn.Dropout(p=0.5),  # Un autre dropout après la deuxième couche cachée
    nn.Linear(512, 28),  # De la deuxième couche cachée à la couche de sortie
    nn.LogSoftmax(dim=1)  # LogSoftmax pour la classification multiclasse
)

learning_rate = 0.01
num_epochs = 20000 

name = 'NN2048-512-28-dropout_Adam'+'_lr_'+str(learning_rate)+'_alpha_5'
print('\n\n Starting to train model', name)
model_trained, Y_pred_probs, Y_pred_tensor, final_score, macro_f1, inv_macro_tpr_gap = train_NN_with_custom_loss(model,optim.Adam(model.parameters(), lr=learning_rate) , 5, X_train_tensor, Y_train_tensor, S_train_tensor, X_test_tensor, Y_test_tensor, S_test_tensor)
Res.loc[i]=[name,optimizer,learning_rate,alpha,final_score, macro_f1, inv_macro_tpr_gap]
save_Y_pred_tofile(X_test_true_tensor, model_trained,name)

In [ ]:
model = nn.Sequential(
    nn.Linear(768, 2048),  # Assuming 768 input features and 28 classes
    nn.ReLU(),  # Adding a ReLU activation function
    nn.Linear(2048,512),  # Assuming 768 input features and 28 classes
    nn.Linear(512,28),  
    nn.Linear(28, 28),  # Additional layer for complexity
    nn.LogSoftmax(dim=1))  # LogSoftmax for multi-class classification

learning_rate = 0.01
num_epochs = 20000 

name = 'NN2048-512-28_Adam'+'_lr_'+str(learning_rate)+'_alpha_5'
print('\n\n Starting to train model', name)
model_trained, Y_pred_probs, Y_pred_tensor, final_score, macro_f1, inv_macro_tpr_gap = train_NN_with_custom_loss(model,optim.Adam(model.parameters(), lr=learning_rate) , 5, X_train_tensor, Y_train_tensor, S_train_tensor, X_test_tensor, Y_test_tensor, S_test_tensor)
Res.loc[i]=[name,optimizer,learning_rate,alpha,final_score, macro_f1, inv_macro_tpr_gap]
save_Y_pred_tofile(X_test_true_tensor, model_trained,name)

In [ ]:
path_pkl = ''

with open(path_pkl + 'RESULTS_11-03-2024.pkl', 'wb') as f:
   pickle.dump(Res, f)

#path_pkl = 'pkl_files/'
#train = pd.read_pickle(path_pkl + 'train_pp.pkl')

**2. REGRESSION WITH CUSTOM LOSS macro F1**
---

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score
import numpy as np

# Assuming model, optimizer, X_train_tensor, Y_train_one_hot, X_test_tensor, Y_test are already defined

# Convert Y_test to one-hot encoding if it's not already one-hot encoded
# This is necessary for consistency in our loss function calculations
Y_test_tensor = torch.tensor(Y_test.values, dtype=torch.int64) if isinstance(Y_test, pd.Series) else torch.from_numpy(Y_test).long()
Y_test_one_hot = torch.nn.functional.one_hot(Y_test_tensor, num_classes=28)


# Define the model using nn.Sequential
model = nn.Sequential(
    nn.Linear(768, 28),  # Assuming 768 input features and 28 classes
    nn.ReLU(),  # Adding a ReLU activation function
    nn.Linear(28, 28),  # Additional layer for complexity
    nn.LogSoftmax(dim=1)  # LogSoftmax for multi-class classification
)

# Define an optimizer
optimizer = optim.SGD(model.parameters(), lr=0.1)

num_epochs = 10000  # Example number of epochs

for epoch in range(num_epochs):
    optimizer.zero_grad()  # Zero the gradients
    
    # Forward pass on the training data
    outputs_train = model(X_train_tensor)
    loss_train = macro_soft_f1_loss(Y_train_one_hot.float(), outputs_train)
    
    # Backward pass and optimize
    loss_train.backward()
    optimizer.step()
    
    # No gradient computation needed for evaluation
    with torch.no_grad():
        model.eval()  # Set the model to evaluation mode
        
        # Forward pass on the validation data
        outputs_test = model(X_test_tensor)
        
        # Calculate the exact macro F1 score for both training and validation data
        f1_train = calculate_exact_macro_f1(Y_train_one_hot.float(), outputs_train)
        f1_test = calculate_exact_macro_f1(Y_test_one_hot.float(), outputs_test)
        
        model.train()  # Set the model back to training mode
    
    # Print loss and F1 score
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss_train.item():.4f}, macro F1 Train: {f1_train:.4f}, macro F1 Test: {f1_test:.4f}')

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report
import torch

# Assuming model is already trained and X_test is a DataFrame

# Convert X_test to a PyTorch tensor
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)

# Make predictions
with torch.no_grad():  # We do not need gradient computation for prediction
    model.eval()  # Set the model to evaluation mode
    Y_pred_probs = model(X_test_tensor)
    Y_pred = torch.argmax(Y_pred_probs, dim=1)  # Get the class with the highest probability

# Convert Y_pred to a DataFrame
Y_pred_df = pd.DataFrame(Y_pred.numpy(), columns=['Predicted'])

# Evaluate Y_pred compared to Y_test (assuming Y_test is a numpy array or a pandas Series)
print(classification_report(Y_test, Y_pred_df['Predicted']))

# If you want to use the exact F1 score for evaluation, you can directly use it from sklearn.metrics
from sklearn.metrics import f1_score
print("Exact F1 Score (micro):", f1_score(Y_test, Y_pred_df['Predicted'],average = 'micro'))  # 'weighted' for multi-class
print("Exact F1 Score (macro):", f1_score(Y_test, Y_pred_df['Predicted'], average='macro'))  # 'weighted' for multi-class

# Returning Y_pred as a DataFrame makes sense for further analysis or submission
#return Y_pred_df

**CUSTON LOSS FUNCTION TRP GAP**
---

In [ ]:
import torch

def gap_TPR(y_true, y_pred, protected_attribute):
    """
    Calculate the average TPR gap for each class across protected groups.
    
    Args:
    - y_true: Tensor of true labels, one-hot encoded.
    - y_pred: Tensor of predicted logits (before softmax).
    - protected_attribute: Tensor indicating group membership for each instance.
    
    Returns:
    - Average TPR gap across all classes.
    """
    # Apply softmax to get probabilities
    y_pred_probs = torch.softmax(y_pred, dim=1)
    
    # Convert one-hot labels to class indices for gathering
    y_true_indices = torch.argmax(y_true, dim=1)
    
    # Initialize TPR storage
    tpr_gaps = []
    
    # Iterate over each class
    num_classes = y_true.shape[1]
    for class_idx in range(num_classes):
        # Calculate TPR for the current class across all groups
        tpr_list = []
        
        # Calculate overall TPR for the current class
        overall_mask = y_true_indices == class_idx
        overall_tpr = torch.sum((y_pred_probs[:, class_idx] > 0.5) & overall_mask).float() / torch.sum(overall_mask).float()
        
        # Calculate TPR for each protected group
        for group_val in protected_attribute.unique():
            group_mask = (protected_attribute == group_val) & overall_mask
            group_tpr = torch.sum((y_pred_probs[:, class_idx] > 0.5) & group_mask).float() / torch.sum(group_mask).float()
            tpr_list.append(group_tpr)
        
        # Calculate TPR gap for the current class and store it
        tpr_gaps.append(torch.abs(torch.tensor(tpr_list) - overall_tpr))
    
    # Calculate the average TPR gap across all classes
    avg_tpr_gap = torch.mean(torch.stack(tpr_gaps))
    
    return avg_tpr_gap

In [ ]:
print(type(Y_test),Y_test.shape)
print(type(Y_pred_probs),Y_pred_probs.shape)
get_macro_tpr_gap(Y_test,Y_pred_probs,S_test)

<class 'pandas.core.series.Series'> (5550,)
<class 'torch.Tensor'> torch.Size([5550, 28])


TypeError: argmax(): argument 'input' (position 1) must be Tensor, not Series